# Compute and compare different radial potentials

In [ ]:
using JAC


We shall here describe how different atomic (radial) potentials can be calculated and applied within the JAC toolbox.

Any (radial) potential is handled in JAC by the data struct `Radial.Potential` that (may) contain a name the radial function and the radial grid on which the potential has been defined. The potential function `Zr` typically contains the effective charge $Z(r)$ as seen by an electron, so that the full radial potential is given by $V = -\:Z(r) / r$. This convention has been frequently applied in atomic physics and has been adopted in JAC.


In [ ]:
? Radial.Potential


Perhaps, the simplest choice is the **nuclear** potential which, of course, depends on the nuclear model.
We consider a $^{12}$C carbon atom with a Fermi-distributed nuclear charge and a standard radial grid.


In [ ]:
grid     = JAC.Radial.Grid("grid: exponential")
nucModel = Nuclear.Model(7., "Fermi")


Then, the nuclear potential is generated by simply calling:


In [ ]:
nucPotential = JAC.Nuclear.nuclearPotential(nucModel, grid)


As seen from the output, the effective charge Z(r) vanishes near to the origin but quickly increases to the full charge Z = 6. at larger r-values, i.e. outside of the nucleus

To calculate **atomic** potentials, we also need to know the electron density around the nuclues. Most often, this density refers to a particular atomic level. Therefore, we first need to compute some of the low-lying levels of, for instance, neutral carbon. We shall here just compute the levels of the 1s^2 2s^2 2p^2 ground-state configuration. We use the *simplest* approximation for these computation but will explain in some later tutorials how we could control and refine such simple structure calculations, if needed.


In [ ]:
basis     = perform("computation: SCF", [Configuration("1s^2 2s^2 2p^2")], nucModel, grid, AsfSettings())
multiplet = perform("computation: CI",  basis, nucModel, grid, AsfSettings());


From this multiplet, we just take the *lowest* level (the ground level) in order to calculate the density for the (electronic) potential. We have moreover different choices to generate such an **electronic** potential as seen from ? compute at the REPL:

    •    ("radial potential: core-Hartree", grid::Radial.Grid, level::Level) ... to compute a (radial) 
           core-Hartree potential from the given list of orbitals; cf. JAC.computePotentialCoreHartree. 
           A potential::RadialPotential is returned. 

    •    ("radial potential: Kohn-Sham", grid::Radial.Grid, level::Level) ... to compute a (radial) 
           Kohn-Sham potential from the given list of orbitals; cf. JAC.computePotentialKohnSham. 
           A potential::RadialPotential is returned.

    •    ("radial potential: Dirac-Fock-Slater", grid::Radial.Grid, level::Level) ... to compute a (radial) 
          Dirac-Fock-Slater potential from the given list of orbitals; this potential is rather simple 
          but includes some undesired self-interaction and exhibits an asymptotic behaviour.         
          Cf. JAC.computePotentialDFS. A potential::RadialPotential is returned.


In [ ]:
level  = multiplet.levels[1]
potCH  = compute("radial potential: core-Hartree", grid, level)
potDFS = compute("radial potential: Dirac-Fock-Slater", grid, level)


All these potentials are represented as *effective charge* Z(r) and can be added to the nuclear potential, for example:

In [ ]:
JAC.add(nucPotential, potDFS)


What can be seen ... and how to display these potentials also graphically; use JAC.plot()